### モデルの作成
---
[参考](http://ni4muraano.hatenablog.com/entry/2017/08/10/101053)

In [ ]:
from keras.models import Model
from keras.layers import Input
from keras.layers.convolutional import Conv2D, ZeroPadding2D, Conv2DTranspose
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose, AveragePooling3D, ZeroPadding3D
from keras.layers.merge import concatenate
from keras.layers import LeakyReLU, BatchNormalization, Activation, Dropout


class DenceUNet(object):
    def __init__(self, input_channel_count, output_channel_count, first_layer_filter_count):
        self.INPUT_IMAGE_SIZE = 512
        self.CONCATENATE_AXIS = -1
        self.CONV_FILTER_SIZE = 4
        self.CONV_STRIDE = 2
        self.CONV_PADDING = (1, 1)
        self.DECONV_FILTER_SIZE = 2
        self.DECONV_STRIDE = 2

        # (512 x 512 x input_channel_count)
        inputs = Input((self.INPUT_IMAGE_SIZE, self.INPUT_IMAGE_SIZE, input_channel_count))
        conv11 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
        conc11 = concatenate([inputs, conv11], axis=3)
        conv12 = Conv2D(32, (3, 3), activation='relu', padding='same')(conc11)
        conc12 = concatenate([inputs, conv12], axis=3)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conc12)

        conv21 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
        conc21 = concatenate([pool1, conv21], axis=3)
        conv22 = Conv2D(64, (3, 3), activation='relu', padding='same')(conc21)
        conc22 = concatenate([pool1, conv22], axis=3)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conc22)

        conv31 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
        conc31 = concatenate([pool2, conv31], axis=3)
        conv32 = Conv2D(128, (3, 3), activation='relu', padding='same')(conc31)
        conc32 = concatenate([pool2, conv32], axis=3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conc32)

        conv41 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
        conc41 = concatenate([pool3, conv41], axis=3)
        conv42 = Conv2D(256, (3, 3), activation='relu', padding='same')(conc41)
        conc42 = concatenate([pool3, conv42], axis=3)
        pool4 = MaxPooling2D(pool_size=(2, 2))(conc42)

        conv51 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
        conc51 = concatenate([pool4, conv51], axis=3)
        conv52 = Conv2D(512, (3, 3), activation='relu', padding='same')(conc51)
        conc52 = concatenate([pool4, conv52], axis=3)

        up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conc52), conc42], axis=3)
        conv61 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
        conc61 = concatenate([up6, conv61], axis=3)
        conv62 = Conv2D(256, (3, 3), activation='relu', padding='same')(conc61)
        conc62 = concatenate([up6, conv62], axis=3)


        up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conc62), conv32], axis=3)
        conv71 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
        conc71 = concatenate([up7, conv71], axis=3)
        conv72 = Conv2D(128, (3, 3), activation='relu', padding='same')(conc71)
        conc72 = concatenate([up7, conv72], axis=3)

        up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conc72), conv22], axis=3)
        conv81 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
        conc81 = concatenate([up8, conv81], axis=3)
        conv82 = Conv2D(64, (3, 3), activation='relu', padding='same')(conc81)
        conc82 = concatenate([up8, conv82], axis=3)

        up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conc82), conv12], axis=3)
        conv91 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
        conc91 = concatenate([up9, conv91], axis=3)
        conv92 = Conv2D(32, (3, 3), activation='relu', padding='same')(conc91)
        conc92 = concatenate([up9, conv92], axis=3)

        conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conc92)

        
        self.DenceUNET = Model(inputs=[inputs], outputs=[conv10])
        

    def _add_encoding_layer(self, filter_count, sequence):
        new_sequence = LeakyReLU(0.2)(sequence)
        new_sequence = ZeroPadding2D(self.CONV_PADDING)(new_sequence)
        new_sequence = Conv2D(filter_count, self.CONV_FILTER_SIZE, strides=self.CONV_STRIDE)(new_sequence)
        new_sequence = BatchNormalization()(new_sequence)
        return new_sequence

    def _add_decoding_layer(self, filter_count, add_drop_layer, sequence):
        new_sequence = Activation(activation='relu')(sequence)
        new_sequence = Conv2DTranspose(filter_count, self.DECONV_FILTER_SIZE, strides=self.DECONV_STRIDE,
                                       kernel_initializer='he_uniform')(new_sequence)
        new_sequence = BatchNormalization()(new_sequence)
        if add_drop_layer:
            new_sequence = Dropout(0.5)(new_sequence)
        return new_sequence

    def get_model(self):
        return self.DenceUNET

### その他関数定義
---

In [ ]:
import cv2
IMAGE_SIZE = 512
TRAIN_PERCENTAGE = 0.8

# 値を-1から1に正規化する関数
def normalize_x(image):
    # image = image/127.5 - 1
    image = (image-np.min(image)) / (np.max(image)/2) - 1
    return image


# 値を0から1に正規化する関数
def normalize_y(image):
    image = image/255
    return image


# 値を0から255に戻す関数
def denormalize_y(image):
    image = image*255
    return image


# インプット画像を読み込む関数
def load_X(folder_path, mode='train'):
    import glob, SimpleITK
    from natsort import natsorted
    
    mhd_files = natsorted(glob.glob(folder_path+'*.mhd'))
    limits = int(len(mhd_files)*TRAIN_PERCENTAGE)

    if mode=='test':
        images = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_files[0]))
        
        image_files = []
        for i in range(images.shape[0]):
            image_files.append(mhd_files[limits].split('/')[-1].split('.')[0] +'-'+str(i))    

        for mhd_name in mhd_files[(int(limits)+1):]:
            mhd_array = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_name))
            images = np.concatenate([images,mhd_array])
            for i in range(mhd_array.shape[0]):
                image_files.append(mhd_name.split('/')[-1].split('.')[0] +'-'+str(i))  
                
    else:
        images = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_files[0]))
        
        image_files = []
        for i in range(images.shape[0]):
            image_files.append(mhd_files[0].split('/')[-1].split('.')[0] +'-'+str(i))    

        for mhd_name in mhd_files[1:int(limits)]:
            mhd_array = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_name))
            images = np.concatenate([images,mhd_array])
            for i in range(mhd_array.shape[0]):
                image_files.append(mhd_name.split('/')[-1].split('.')[0] +'-'+str(i))     
    
    images = normalize_x(images[:, :, :, np.newaxis])
    return images, image_files


# ラベル画像を読み込む関数
def load_Y(folder_path):
    import glob, SimpleITK
    from natsort import natsorted
    
    mhd_files = natsorted(glob.glob(folder_path+'*/*[!2].mhd'))
    limits = int(len(mhd_files)*TRAIN_PERCENTAGE)
    
    images = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_files[0]))
    for mhd_name in mhd_files[1:int(limits)]:
        mhd_array = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_name))
        images = np.concatenate([images,mhd_array])
    
    # GTの値の範囲が0-6だったので、1以上は1にしている
    images = np.where(images >= 1 , 1 ,images)
    return images[:, :, :, np.newaxis]


### train/test部分
---

参考：
- [クロスバリデーション](https://lp-tech.net/articles/Y56uo)
- [モデルチェックポイント](https://blog.shikoan.com/keras-model-checkpoint-save-best-only/)
- [その全て](https://keras.io/ja/callbacks/)

In [ ]:
from keras import backend as K

# GPU1つのみの設定
if 'tensorflow' == K.backend():
    import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = "1"
set_session(tf.Session(config=config))

In [ ]:
import os
import numpy as np
from keras.optimizers import Adam
import keras.backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

# from unet import UNet

# ダイス係数を計算する関数
def dice_coef(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)
    return 2.0 * intersection / (K.sum(y_true) + K.sum(y_pred) + 1)


# ロス関数
def dice_coef_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)


# denceU-Netのトレーニングを実行する関数
def train_denceunet():
    # trainingDataフォルダ配下にleft_imagesフォルダを置いている
    X_train, file_names = load_X('../Preprocess/mhd/', 'train')
    # trainingDataフォルダ配下にleft_groundTruthフォルダを置いている
    Y_train = load_Y('../Preprocess/GroundTruth/')

    # 入力はBGR3チャンネル
    input_channel_count = 1
    # 出力はグレースケール1チャンネル
    output_channel_count = 1
    # 一番初めのConvolutionフィルタ枚数は64
    first_layer_filter_count = 64

    # denceU-Netの生成
    network = DenceUNet(input_channel_count, output_channel_count, first_layer_filter_count)
    model = network.get_model()
    model.compile(optimizer=Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.000000199), metrics=[dice_coef], loss=dice_coef_loss )
    # plot_model(model, to_file='model.png')
    
    model.summary()

    BATCH_SIZE = 1
    # 20エポック回せば十分
    NUM_EPOCH = 20
    
        
    kf = KFold(n_splits=15, shuffle=False)
    
    for i, (train_index, eval_index) in enumerate(kf.split(X_train)):
        X_tra, X_eval = X_train[train_index], X_train[eval_index]
        y_tra, y_eval = Y_train[train_index], Y_train[eval_index]
    
        cp = ModelCheckpoint("weights/weights-"+str(i)+".{epoch:02d}-{val_loss:.2f}.hdf5",
                             monitor="val_loss", verbose=1, save_best_only=True, save_weights_only=False)
        learning_history = model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCH, callbacks=[cp], validation_data=(X_eval, y_eval))


# 学習後のdenceU-Netによる予測を行う関数
def predict():
    import cv2

    # testDataフォルダ配下にleft_imagesフォルダを置いている
    X_test, file_names = load_X('../Preprocess/mhd/', 'test')

    input_channel_count = 1
    output_channel_count = 1
    first_layer_filter_count = 64
    network = DenceUNet(input_channel_count, output_channel_count, first_layer_filter_count)
    model = network.get_model()
    model.load_weights('denceunet_weights.hdf5')
    BATCH_SIZE = 12
    Y_pred = model.predict(X_test, BATCH_SIZE)
    
    # 変な方向の断面図でないか、気になる！
    print(Y_pred.shape)
    
    # testDataフォルダ配下にleft_imagesフォルダを置いている
    save_path = 'DenceU-Net_DataSet/pred/'
    if not os.path.exists(save_path): os.makedirs(save_path)

    for i, y in enumerate(Y_pred):
        #img = cv2.imread(save_path + file_names[i]+".png")
        #y = cv2.resize(y, (img.shape[1], img.shape[0]))
        cv2.imwrite(save_path+'prediction' + file_names[i] + '.png', denormalize_y(y))


#if __name__ == '__main__':
#    train_denceunet()
#     predict()

In [ ]:
train_denceunet()

In [ ]:
learning_history